In [ ]:
import os
import re
import time
import math
import random

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [ ]:
driver = webdriver.Chrome()

In [ ]:
# Districts are from: https://www.doogal.co.uk/london_postcodes
districts = [f'EC{i}{l}' for i in range(1,5) for l in ('A','M','N','P','R','V')]\
    + [f'EC{i}Y' for i in (1,2,4)]\
    + [f'WC{i}{l}' for i in (1,2) for l in ('A','B','E','H','N','R')]\
    + ['WC1V','WC1X','E20']\
    + [f'{l}{i}' for m,l in [(19,'E'),(23,'N'),(12,'NW'),(29,'SE'),(21,'SW'),(15,'W')] for i in range(1,m)]
print(f'Number of districts: {len(districts)}')
print('District list:', districts)

In [ ]:
def scrape_schools(
    url : str,
    save_folder : str
) -> None:
    for district in districts:
        driver.get(url.format(district))
        not_finished = True
        time.sleep(random.uniform(5,8))
        # Finish when 1) the page shows any other district schools or 2) when reached the last page 
        while not_finished:
            page_html = driver.page_source
            # Parse page HTML
            page = BeautifulSoup(page_html, 'html.parser')
            schools = page.find_all('ul', class_='results-list')[0]
            schools_list = []
            for school in schools.find_all('li', class_='search-result'):
                school_info = []
                s = school.find('h3').find('a')
                # School name
                school_info.append(s.get_text().strip())
                # School link
                school_info.append(s['href'])
                # School address
                school_info.append(school.find('address').get_text().strip())
                sub_info = school.find_all('ul')
                # School category
                school_info.append(sub_info[0].find('strong').get_text().strip())
                metric_type = sub_info[1].get_text().split(':')[0]
                # Newly registered schools don't have any further info
                if 'a newly registered school' in metric_type:
                    school_info.append('')
                    school_info.append('')
                else:
                    # School last inspection score
                    if metric_type == 'Rating':
                        school_info.append(sub_info[1].find('strong').get_text().strip())
                        report_index = 2
                    # Some schools are not rated yet
                    else:
                        school_info.append('')
                        report_index = 1
                    # School last report date
                    school_info.append(sub_info[report_index].find('time').get_text().strip())
                    # School URN
                    school_info.append(sub_info[report_index].find_all('strong')[1].get_text().strip())
                schools_list.append(school_info)
            schools_df = pd.DataFrame(schools_list, columns=['school_name','link','address','category','rating','last_report_date','urn'])
            # If all schools on the page are in the target district, move to the next page
            if schools_df.shape[0] == schools_df[schools_df['address'].str.contains(district)].shape[0]:
                try:
                    driver.find_element(By.XPATH, "//a[@class='pagination__next']").click()
                    time.sleep(random.uniform(5,8))
                except:
                    not_finished = False
            else:
                not_finished = False
        schools_df = pd.DataFrame(schools_list, columns=['school_name','link','address','category','rating','last_report_date','urn'])
        schools_df.to_csv(f'{save_folder}/{district}.csv',index=False)

In [ ]:
# This URL has the following selection criteria:
# 1) Chosen London district
# 2) Radius = 3 miles
# 3) Category = 'Education and Training'
# 4) Sub-category = 'Primary' (only primary schools)
# 5) Status = Open
# 6) Query starts at 0
# 7) Query should show 100 results
url_all = "https://reports.ofsted.gov.uk/search?q=&location=London+{}%2C+UK&radius=1&radius=3&level_1_types=1&level_2_types=1&latest_report_date_start=&latest_report_date_end=&status%5B%5D=1&start=0&rows=100"
save_folder = 'data/schools'
scrape_schools(url_all,save_folder)

In [ ]:
data_all_schools = []
for district in districts:
    data_all_schools.append(pd.read_csv(f'data/schools/{district}.csv'))
# Concatenate all London schools and drop duplicates
# We overscrape - one district query shows queries of other districts as well
# Rating and URN have NaNs or might be of multiple types (string, int, float), so we don't consider them in duplication removal
# This exclusion does not cause any unique removals 
data_all_schools = pd.concat(data_all_schools).drop_duplicates(['school_name', 'link', 'address', 'category','last_report_date']).reset_index(drop=True)
data_all_schools.to_csv(f'data/all_schools.csv',index=False)

In [ ]:
# This URL has the same selection criteria as the selection of all schools
# Plus, it selects all available religions in 'Religion' filter tab
url_religious = "https://reports.ofsted.gov.uk/search?q=&location=London+{}%2C+UK&radius=1&radius=3&level_1_types=1&level_2_types=1&religion%5B%5D=2&religion%5B%5D=3&religion%5B%5D=4&religion%5B%5D=5&religion%5B%5D=6&religion%5B%5D=7&religion%5B%5D=8&religion%5B%5D=9&religion%5B%5D=10&religion%5B%5D=11&religion%5B%5D=12&religion%5B%5D=13&religion%5B%5D=14&religion%5B%5D=15&religion%5B%5D=16&religion%5B%5D=17&religion%5B%5D=18&religion%5B%5D=19&religion%5B%5D=20&latest_report_date_start=&latest_report_date_end=&status%5B%5D=1&start=0&rows=100"
save_folder_religious = 'data/religious_schools'
scrape_schools(url_religious,save_folder_religious)

In [ ]:
data_religious_schools = []
for district in districts:
    data_religious_schools.append(pd.read_csv(f'data/religious_schools/{district}.csv'))
data_religious_schools = pd.concat(data_religious_schools).drop_duplicates(['school_name', 'link', 'address', 'category','last_report_date']).reset_index(drop=True)
data_religious_schools.to_csv(f'data/religious_schools.csv',index=False)

In [ ]:
# Add is_religion feature to all schools dataset
data_religious_schools['is_religious'] = True 
data_all_schools = pd.merge(data_all_schools, data_religious_schools[['link','is_religious']], how='left',on=['link'])
data_all_schools.loc[data_all_schools['is_religious'].isna(), 'is_religious'] = False
# Some schools are not labeled as religious, but contain saints in the names, so change label for them
data_all_schools.loc[data_all_schools['school_name'].str.contains('\s?St '), 'is_religious'] = True
data_all_schools.to_csv(f'data/all_schools.csv',index=False)

In [ ]:
pd.read_csv('data/all_schools.csv')